In [ ]:
!rm -rf t5-lora-out
!rm -rf t5-small-lora-adapter

In [ ]:
!pip install -q transformers datasets peft accelerate

In [ ]:
import pandas as pd
from datasets import Dataset
df = pd.read_csv("customer_support_lora_dataset_250.csv")
ds = Dataset.from_pandas(df)
print(ds[0])

In [ ]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq

tokenizer = AutoTokenizer.from_pretrained("t5-small")

def preprocess(ex):
    model_inputs  = tokenizer(ex["input_text"],  max_length=128, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(ex["target_text"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

ds_tok = ds.map(preprocess, remove_columns=ds.column_names)
print(ds_tok[0].keys(), "→ lengths:", len(ds_tok[0]["input_ids"]), len(ds_tok[0]["labels"]))


In [ ]:
from transformers import AutoModelForSeq2SeqLM
from peft import LoraConfig, get_peft_model, TaskType

base_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

lora_cfg = LoraConfig(
    task_type      = TaskType.SEQ_2_SEQ_LM,
    r              = 8,
    lora_alpha     = 16,
    target_modules = ["q", "v"],
    lora_dropout   = 0.05,
    bias           = "none"
)

peft_model = get_peft_model(base_model, lora_cfg)
peft_model.print_trainable_parameters()


In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

training_args = Seq2SeqTrainingArguments(
    output_dir          = "./t5-lora-out",
    per_device_train_batch_size = 16,
    num_train_epochs    = 30,
    learning_rate       = 5e-4,
    logging_steps       = 5,
    save_strategy       = "no",
    report_to           = "none",
)

trainer = Seq2SeqTrainer(
    model         = peft_model,
    args          = training_args,
    train_dataset = ds_tok,
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=peft_model),
)

trainer.train()


In [ ]:
trainer.model.save_pretrained("t5-small-lora-adapter") 
tokenizer.save_pretrained("t5-small-lora-adapter")


In [ ]:
from peft import PeftConfig, PeftModel
from transformers import AutoModelForSeq2SeqLM

cfg         = PeftConfig.from_pretrained("t5-small-lora-adapter")
base_model  = AutoModelForSeq2SeqLM.from_pretrained(cfg.base_model_name_or_path)
model_lora  = PeftModel.from_pretrained(base_model, "t5-small-lora-adapter")

prompt = "generate reply: My order arrived late. I want a refund."
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model_lora.generate(**inputs, max_new_tokens=80)

print("→", tokenizer.decode(outputs[0], skip_special_tokens=True))


In [ ]:

# Compare vanilla T5-small with LoRA fine‑tuned adapter on structured JSON output
import json, pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftConfig, PeftModel

tokenizer = AutoTokenizer.from_pretrained("t5-small")

# Base (pre‑trained) model
base_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

# LoRA‑adapted model – make sure this path matches the one used in the training cell
adapter_path = "t5-small-lora-adapter"
cfg       = PeftConfig.from_pretrained(adapter_path)
ft_model  = PeftModel.from_pretrained(
    AutoModelForSeq2SeqLM.from_pretrained(cfg.base_model_name_or_path),
    adapter_path
)

def generate(model, prompt):
    ids = tokenizer(prompt, return_tensors="pt")
    out = model.generate(**ids, max_new_tokens=120)
    return tokenizer.decode(out[0], skip_special_tokens=True)

test_prompts = [
    "generate structured_reply: My order arrived late. I want a refund.",
    "generate structured_reply: The product I received is damaged..",
    "generate structured_reply: I correctly received my order. Thanks!"
]
test_prompts = [
    "generate structured_reply: My order arrived late. I want a refund.",
    "generate structured_reply: The product I received is damaged. What can I do?",
    "generate structured_reply: I received the wrong item in my order.",
    "generate structured_reply: How can I return an item I purchased last week?",
    "generate structured_reply: I never received my order.",
    "generate structured_reply: Why was I charged twice for my order?",
    "generate structured_reply: I need help tracking my shipment.",
    "generate structured_reply: Can I exchange my item for a different size?"
]

records = []
for p in test_prompts:
    records.append({
        "prompt": p,
        "T5-base": generate(base_model, p),
        "LoRA":    generate(ft_model,  p)
    })

df = pd.DataFrame(records)
print(df.to_markdown(index=False))
